In [1]:
# Cell 1: Import necessary libraries for data conversion
import pandas as pd
import numpy as np
import cv2 # OpenCV for image saving
import os # For directory operations

# Cell 2: Load the fer2013.csv dataset
# Make sure 'fer2013.csv' is in the same directory as your Jupyter Notebook
try:
    data = pd.read_csv('fer2013.csv')
    print("fer2013.csv loaded successfully.")
    print("Dataset head:\n", data.head())
    print("\nDataset info:\n")
    data.info()
except FileNotFoundError:
    print("ERROR: fer2013.csv not found. Please ensure it's in the same directory as this notebook.")
    # Exit or handle the error gracefully
    exit() # You might remove this for interactive testing, but it will stop execution.

# Cell 3: Define constants and output directories
# Emotion labels mapping from integer to string
emotion_labels = {
    0: 'angry',
    1: 'disgust',
    2: 'fear',
    3: 'happy',
    4: 'sad',
    5: 'surprise',
    6: 'neutral'
}

# Base directory where all organized images will be saved
base_output_dir = 'fer2013_images_organized'

# Create the main output directory
os.makedirs(base_output_dir, exist_ok=True)

# Define subdirectories for training, validation, and (optional) test sets
# FER2013 CSV uses 'Training', 'PublicTest', 'PrivateTest'
# We'll map them to 'train', 'validation', 'test'
usage_mapping = {
    'Training': 'train',
    'PublicTest': 'validation',
    'PrivateTest': 'test'
}

# Create all necessary subdirectories
for usage_type in usage_mapping.values():
    usage_path = os.path.join(base_output_dir, usage_type)
    os.makedirs(usage_path, exist_ok=True)
    for emotion_name in emotion_labels.values():
        emotion_path = os.path.join(usage_path, emotion_name)
        os.makedirs(emotion_path, exist_ok=True)
print(f"Directory structure created at: {base_output_dir}")


# Cell 4: Convert CSV pixels to image files and save them
print("Starting image conversion and saving...")
image_count = 0
for index, row in data.iterrows():
    try:
        # Convert pixel string to numpy array and reshape to 48x48
        pixels_str = row['pixels'].split()
        pixels = np.array(pixels_str, dtype='uint8').reshape(48, 48)

        emotion_id = row['emotion']
        usage_type_raw = row['Usage']

        emotion_name = emotion_labels[emotion_id]
        
        # Get the target directory name (e.g., 'train', 'validation', 'test')
        save_dir_name = usage_mapping.get(usage_type_raw)
        
        if save_dir_name is None:
            print(f"Warning: Skipping row {index} with unrecognized usage type: {usage_type_raw}")
            continue

        # Construct the full path to save the image
        output_folder = os.path.join(base_output_dir, save_dir_name, emotion_name)
        filename = f"{emotion_name}_{index}.png" # Use PNG for lossless grayscale
        filepath = os.path.join(output_folder, filename)

        # Save the grayscale image
        cv2.imwrite(filepath, pixels)
        image_count += 1
        
        if image_count % 1000 == 0:
            print(f"Processed {image_count} images...")

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue

print(f"Image conversion complete. Total images saved: {image_count}")
print(f"Your images are now organized in the '{base_output_dir}' directory.")

fer2013.csv loaded successfully.
Dataset head:
    emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training

Dataset info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
Directory structure created at: fer2013_images_organized
Starting image conversion and saving...
Processed 1000 images...
Processed 2000 images...
Processed 30

In [4]:
# Cell 5: Import necessary libraries for model training and data generation
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# No need to import Sequential, Conv2D, etc. here, as we'll do it in the next step (model building)
import numpy as np
import matplotlib.pyplot as plt # For plotting training history
import os # For directory paths


# Define image dimensions and batch size
IMG_HEIGHT = 48
IMG_WIDTH = 48
BATCH_SIZE = 64 # Adjust this based on your system's memory/GPU. Smaller batches use less memory.

# Define the base directory where your organized images are located
# This should match the 'base_output_dir' from Step 2
base_output_dir = 'fer2013_images_organized' # Ensure this matches

train_dir = os.path.join(base_output_dir, 'train')
validation_dir = os.path.join(base_output_dir, 'validation')
test_dir = os.path.join(base_output_dir, 'test') # Use if you want to evaluate on test set

print(f"Training images path: {train_dir}")
print(f"Validation images path: {validation_dir}")
print(f"Test images path: {test_dir}") # If applicable

Training images path: fer2013_images_organized\train
Validation images path: fer2013_images_organized\validation
Test images path: fer2013_images_organized\test


In [5]:
# Cell 6: Configure ImageDataGenerators and create data generators

# ImageDataGenerator for Training Data (with Data Augmentation)
# Rescale: Normalizes pixel values from [0, 255] to [0, 1]
# Data Augmentation parameters: These randomly transform images to make the model more robust
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,        # Rotate images by up to 10 degrees
    width_shift_range=0.1,    # Shift images horizontally by up to 10%
    height_shift_range=0.1,   # Shift images vertically by up to 10%
    shear_range=0.1,          # Apply shear transformations
    zoom_range=0.1,           # Apply zoom transformations
    horizontal_flip=True,     # Randomly flip images horizontally
    fill_mode='nearest'       # Strategy for filling in new pixels created by transformations
)

# ImageDataGenerator for Validation and Test Data (only Rescaling)
# Validation and test data should NOT be augmented, only scaled, to ensure realistic evaluation
validation_test_datagen = ImageDataGenerator(rescale=1./255)


# Create data generators for training and validation sets
# flow_from_directory reads images from the organized folders
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH), # Resize all images to 48x48
    batch_size=BATCH_SIZE,
    color_mode='grayscale', # Our images are grayscale (1 channel)
    class_mode='categorical' # Labels are one-hot encoded (e.g., [0,0,1,0,0,0,0])
)

validation_generator = validation_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical'
)

# Optional: Create a generator for the test set (recommended for final evaluation)
test_generator = validation_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False # Keep data in order for consistent evaluation
)


# Get the class indices and map them to emotion labels
# This dictionary maps integer indices (used by the model) back to emotion names (folder names)
# It's good to re-establish this map based on the generator's internal mapping
emotion_labels_map = {v: k for k, v in train_generator.class_indices.items()}
print("\nClass indices from generator:", train_generator.class_indices)
print("Emotion labels mapping (integer to name):", emotion_labels_map)

# Get the number of classes (emotions)
num_classes = len(emotion_labels_map)
print("Number of classes detected:", num_classes)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.

Class indices from generator: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
Emotion labels mapping (integer to name): {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
Number of classes detected: 7


In [6]:
# Cell 7: Build the CNN Model Architecture

# Import layers specific to model building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

model = Sequential()

# Input Layer and first Convolutional Block
# Conv2D: Learns features from the image (32 filters, 3x3 kernel size)
# Input shape: (IMG_HEIGHT, IMG_WIDTH, 1) - 48x48 pixels, 1 channel for grayscale
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)))
model.add(BatchNormalization()) # Helps stabilize and speed up training
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) # Reduces spatial dimensions, makes model robust to small shifts
model.add(Dropout(0.25)) # Randomly drops neurons to prevent overfitting

# Second Convolutional Block
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third Convolutional Block
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten Layer: Converts the 2D feature maps into a 1D vector
model.add(Flatten())

# Dense (Fully Connected) Layers
model.add(Dense(256, activation='relu')) # A large layer to integrate features
model.add(BatchNormalization())
model.add(Dropout(0.5)) # More aggressive dropout to prevent overfitting on the dense layers

# Output Layer
# num_classes: Number of distinct emotions (e.g., 7)
# softmax activation: Outputs probabilities for each class, summing to 1
model.add(Dense(num_classes, activation='softmax'))

# Compile the Model
# optimizer: Adam is a popular and effective optimizer
# loss: categorical_crossentropy is used for multi-class classification with one-hot encoded labels
# metrics: 'accuracy' to monitor performance during training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary - useful to see the layers, output shapes, and number of parameters
model.summary()

C:\Users\Adarsha\miniconda3\envs\emotion_detector_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 48, 48, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 48, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 48, 48, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 48, 48, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 24, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 24, 24, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 24, 24, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 24, 24, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 24, 24, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 12, 12, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 12, 12, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 12, 12, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,470,951 (5.61 MB)

 Trainable params: 1,469,543 (5.61 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [ ]:
# Cell 8: Train the Model

# Define Callbacks (Highly Recommended!)

# ModelCheckpoint: Saves the best model weights based on validation accuracy
checkpoint = ModelCheckpoint(
    'emotion_model_best.h5', # File name to save the model
    monitor='val_accuracy', # Monitor validation accuracy
    mode='max', # Save when val_accuracy is maximized
    save_best_only=True, # Only save if the current model is better than previous best
    verbose=1 # Show messages when saving
)

# EarlyStopping: Stops training if validation accuracy doesn't improve for a certain number of epochs
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=20, # Number of epochs with no improvement after which training will be stopped
    mode='max',
    verbose=1,
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored quantity.
)

# ReduceLROnPlateau: Reduces learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', # Monitor validation loss
    factor=0.2, # Reduce learning rate by 20%
    patience=5, # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,
    min_lr=0.00001 # Minimum learning rate
)

callbacks_list = [checkpoint, early_stopping, reduce_lr]

# Train the Model
# epochs: Number of complete passes through the training dataset
# steps_per_epoch: Total number of batches (steps) to draw from the generator before declaring one epoch finished.
#                   Calculated as total_training_samples // batch_size
# validation_data: The generator for validation data
# validation_steps: Total number of batches to draw from the validation generator.
#                    Calculated as total_validation_samples // batch_size
print("\nStarting model training. This may take a while...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=100, # Start with a reasonable number, EarlyStopping will prevent unnecessary training
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=callbacks_list,
    verbose=1 # Show training progress
)

print("\nTraining finished!")

# Cell 9: Optional: Evaluate the model on the test set and Plot Training History

# Evaluate the model on the test set if the generator was created
# This gives you an unbiased estimate of the model's performance on unseen data
if 'test_generator' in locals() and test_generator is not None:
    print("\nEvaluating model on the test set...")
    # Calculate steps for the test generator
    test_steps = test_generator.samples // BATCH_SIZE
    if test_generator.samples % BATCH_SIZE != 0: # Add one more step if there's a remainder batch
        test_steps += 1
    
    loss, accuracy = model.evaluate(test_generator, steps=test_steps, verbose=1)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

# Plot training & validation accuracy values
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()


Starting model training. This may take a while...


C:\Users\Adarsha\miniconda3\envs\emotion_detector_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
 12/448 ━━━━━━━━━━━━━━━━━━━━ 5:53 810ms/step - accuracy: 0.1668 - loss: 2.9791

In [2]:
# Cell 10: Live Emotion Detection using Webcam

# Import necessary libraries
import cv2
import numpy as np
from tensorflow.keras.models import load_model # To load your trained model
import os # For checking file paths

# Define the path to your trained model
model_path = 'emotion_model_best.h5'

# Load the trained emotion detection model
try:
    model = load_model(model_path)
    print(f"Model '{model_path}' loaded successfully.")
except Exception as e:
    print(f"Error loading model from {model_path}: {e}")
    print("Please ensure 'emotion_model_best.h5' is in the same directory as this notebook.")
    exit() # Exit if model cannot be loaded

# Load the Haar Cascade classifier for face detection
# You'll need 'haarcascade_frontalface_default.xml'.
# This path is common for Miniconda/Anaconda installations.
# If it fails, you might need to find the exact path on your system.
# Search your Anaconda/Miniconda installation for 'haarcascade_frontalface_default.xml'
# e.g., C:\Users\YourUser\Miniconda3\envs\your_env_name\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml
# Or download it directly from OpenCV's GitHub: https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_classifier_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

# Check if the cascade file exists
if not os.path.exists(face_classifier_path):
    print(f"ERROR: Face cascade classifier not found at {face_classifier_path}")
    print("Please check the path or download 'haarcascade_frontalface_default.xml' and place it in a reachable location.")
    exit() # Exit if cascade file cannot be found

face_classifier = cv2.CascadeClassifier(face_classifier_path)

# Define emotion labels (ensure this matches your model's output mapping)
# Use the emotion_labels_map generated from train_generator.class_indices in Cell 6
# It's crucial that these match the order your model was trained on.
# You can get this from the output of Cell 6 (emotion_labels_map)
# For example, if your output was {0: 'angry', 1: 'disgust', ...}, use that.
# Let's assume the default FER2013 order:
emotion_labels = {
    0: 'angry',
    1: 'disgust',
    2: 'fear',
    3: 'happy',
    4: 'sad',
    5: 'surprise',
    6: 'neutral'
}

# Ensure the emotion_labels_map from your generator is consistent
# If Cell 6 outputted something different, update this `emotion_labels` dict.
# For example, if train_generator.class_indices was {'angry':0, 'disgust':1, ...}
# then emotion_labels_map = {0: 'angry', 1: 'disgust', ...}
# It's safer to directly use emotion_labels_map if you ran Cell 6.
# If you want to use the exact mapping, replace the above `emotion_labels` dict with:
# emotion_labels = emotion_labels_map


# Start webcam capture
cap = cv2.VideoCapture(0) # 0 is usually the default webcam

if not cap.isOpened():
    print("Error: Could not open webcam. Make sure it's connected and not in use by another application.")
    exit()

print("Starting live emotion detection. Press 'q' to quit.")

while True:
    ret, frame = cap.read() # Read a frame from the webcam
    if not ret:
        print("Failed to grab frame.")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert frame to grayscale
    
    # Detect faces in the grayscale frame
    # scaleFactor: How much the image size is reduced at each image scale
    # minNeighbors: How many neighbors each candidate rectangle should have to retain it
    # minSize: Minimum possible object size. Objects smaller than this are ignored.
    faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) # Draw rectangle around face

        # Extract the Region of Interest (ROI) - the face
        roi_gray = gray_frame[y:y + h, x:x + w]
        
        # Resize the ROI to 48x48 pixels (model input size)
        # INTER_AREA is good for shrinking images
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0: # Check if the ROI is not completely black (i.e., actually contains a face)
            # Preprocess the ROI for model prediction
            roi = roi_gray.astype('float') / 255.0 # Normalize pixel values to [0, 1]
            roi = np.expand_dims(roi, axis=0) # Add batch dimension (1, 48, 48)
            roi = np.expand_dims(roi, axis=-1) # Add channel dimension for grayscale (1, 48, 48, 1)

            # Make a prediction using the loaded model
            prediction = model.predict(roi)[0] # Get the probability distribution for emotions
            
            # Get the index of the emotion with the highest probability
            emotion_idx = np.argmax(prediction)
            
            # Get the emotion text label
            emotion_text = emotion_labels[emotion_idx]
            
            # Get the probability for the detected emotion
            probability = prediction[emotion_idx] * 100
            
            # Prepare text to display
            display_text = f"{emotion_text}: {probability:.2f}%"

            # Put the text on the frame
            cv2.putText(frame, display_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
        else:
            # If no meaningful pixels in ROI, display "No Face" (unlikely for a detected face)
            cv2.putText(frame, 'No Face', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Live Emotion Detector', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()
print("Webcam feed stopped.")

Model 'emotion_model_best.h5' loaded successfully.
Starting live emotion detection. Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━